In [1]:
!pip install googletrans==4.0.0rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 1.4 MB 11.5 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17414 sha256=1e613e3ca319741311b5890fe7ba2eecf9599f6420104947760fa308bfb90bed
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [2]:
%cd /content/drive/MyDrive/Colab\ Notebooks/NMT\ Dataset\ Augmentation

/content/drive/MyDrive/Colab Notebooks/NMT Dataset Augmentation


In [3]:
from googletrans import Translator
from tqdm import tqdm
import re

<h3><i>1- Download text data</i></h3>

In [4]:
file_name = "the_short_history_of_the_world.txt"
url = "https://www.gutenberg.org/cache/epub/35461/pg35461.txt"
# A short history of the world

!curl {url} --output {file_name}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  739k  100  739k    0     0  1708k      0 --:--:-- --:--:-- --:--:-- 1708k


<h3><i>2- Read file.</i></h3>

In [5]:
with open(file_name, "r") as file:
    text = file.read()

print(len(text))

738419


<h3><i>3- Preprocess raw text.</i></h3>

In [6]:
split_mark = """



II
THE WORLD IN TIME
"""

def clean_raw_text(text: str)-> str:
    text = text.split(split_mark)     # split file header from book content.
    text = text[1]                    # using just book content.
    text = text.replace("\n", " ")    # repalce new line token.
    text = text.replace("\t", " ")    # replace tab token.
    text = text.strip()               # remove extra white spaces at beginig and at the end of the text
    return text


def clean_sentence(sentence: str)-> str:
    sentence = re.sub(r"[^A-Za-z, ]", "", sentence)      # remove anything except whitespaces and charecters.
    sentence = re.sub(' +', ' ', sentence)              # remove extra whitespaces.
    sentence = sentence.strip()                         # remove extra white spaces at beginig and at the end of the text.
    return sentence

In [7]:
whole_text = clean_raw_text(text=text)   # clean raw text
sentences = whole_text.split(".")       # split text into sentences

clean_sentences = []
for s in tqdm(sentences):
    cleaned_sentence = clean_sentence(s)
    clean_sentences.append(cleaned_sentence)

print("done .")

100%|██████████| 6112/6112 [00:00<00:00, 97848.36it/s]

done .


<h3><i>4- Filter sentences based on length.</i></h3>

In [13]:
min_len = 4
max_len = 18
num_instances = 2500

desired_sentences = []
for s in clean_sentences:
    if len(s.split()) >= min_len and len(s.split()) <= max_len:
        desired_sentences.append(s)
    
    if len(desired_sentences) >= num_instances:
        break

print(len(desired_sentences))

2499


<h3><i>5- Translate.</i></h3>

In [16]:
translator = Translator()

translated_text = []
for i, s in enumerate(tqdm(desired_sentences)):
    translated = translator.translate(s, dest="persian").text
    translated_text.append(translated)
    if i+1 != len(translated_text):
        print(f"error at index no: {i}")
        break

100%|██████████| 2499/2499 [43:33<00:00,  1.05s/it]


<h3><i>6- Save translated.</i></h3>

In [19]:
delimiter = "\n"

english_sentences = delimiter.join(desired_sentences)
persian_sentences = delimiter.join(translated_text)

In [22]:
english_path = "english_sentences.txt"
persian_path = "persian_sentences.txt"

with open(english_path, "a") as file:
    file.write(english_sentences)


In [23]:
with open(persian_path, "a") as file:
    file.write(persian_sentences)